In [1]:
%load_ext autoreload
%autoreload 2
import spike_queries
import pandas as pd
import tqdm
import pickle
import random
import itertools

import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel, AutoTokenizer, AutoModel, PreTrainedTokenizerFast
import numpy as np
from typing import List
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from typing import Dict, Tuple
from scipy.spatial.distance import cosine as cosine_distance
from collections import defaultdict
from nltk import ngrams as get_ngrams
from termcolor import colored

2020-11-23 09:15:57.080 INFO    faiss: Loading faiss with AVX2 support.


In [2]:
with open("queries2.txt", "r") as f:
    queries = f.readlines()
    queries = [l.strip() for l in queries]


In [3]:
# for i,q in enumerate(queries):
#     try:
#         assert len(q.split("\t")) == 2
#     except:
#         print(i, q, len(q.split("\t")))

In [4]:
# for i,q in enumerate(queries):
#     a = q.count("arg1:") == q.count("arg2:") == 1
#     if not a:
#         print(i,q)

In [5]:
# queries2results = defaultdict(list)

In [6]:
# dataset_name = "covid19"
# num_results = 100
# query_type = "syntactic"

# for i, q_and_id in tqdm.tqdm(enumerate(queries), total = len(queries)):
#     q, id = q_and_id.split("\t")
#     id = int(id)
#     try:
#         df = spike_queries.perform_query(q, dataset_name, num_results, query_type) #previously: word=Hawaii
#         queries2results[id].append(df)
#         time.sleep(2)
#     except Exception as e:
#         print("Error", i+1)
#         print(e)
        
#         continue

In [7]:
# for id, dfs in queries2results.items():
#     for i,df in enumerate(dfs):
#         print(id, i)
#         queries2results[id][i] = df.dropna(subset=['arg1_first_index',"arg1_last_index","arg2_first_index","arg2_last_index"])

In [8]:
# len(queries2results)

In [9]:
# with open("queries2results.pickle", "wb") as f:
#      pickle.dump(queries2results, f)

In [10]:
with open("queries2results.pickle", "rb") as f:
    queries2results = pickle.load(f)

In [11]:
# queries2results.keys()

In [12]:
# queries2results_new = dict()
# for k,dfs in queries2results.items():
#     if k == 0: continue
#     queries2results_new[k] = pd.concat(dfs, axis=0)
    
# queries2results = queries2results_new

In [13]:
# df = queries2results[12]
# len(df)
# df2 = df.dropna(subset=['arg1_first_index',"arg1_last_index","arg2_first_index","arg2_last_index"])
# df2["arg2_first_index"]

In [14]:
# ids = set([int(l.split("\t")[-1]) for l in queries])

In [15]:
# data = []
# for id in ids:
#     results = queries2results[id]
#     sents, arg1_first, arg2_first = results["sentence_text"].tolist(), results["arg1_first_index"].tolist(), results["arg2_first_index"].tolist()
#     arg1_last, arg2_last = results["arg1_last_index"].tolist(), results["arg2_last_index"].tolist()
    
#     sents_with_args = []
#     for s,arg1_ind,arg2_ind, arg1_ind_last, arg2_ind_last in zip(sents,arg1_first,arg2_first, arg1_last, arg2_last):
#         s_lst = s.split(" ")
#         if arg1_ind > arg2_ind:
#             arg1_ind, arg2_ind = arg2_ind, arg1_ind
#             arg1_ind_last, arg2_ind_last = arg2_ind_last, arg1_ind_last
#             arg1_str, arg2_str = "ARG2:", "ARG1:"
#         else:
#             arg1_str, arg2_str = "ARG1:", "ARG2:"
#         s_with_args = s_lst[:arg1_ind] + [arg1_str+s_lst[arg1_ind]] + s_lst[arg1_ind+1:arg2_ind] + [arg2_str+s_lst[arg2_ind]] + s_lst[arg2_ind+1:]
#         s_with_args = " ".join(s_with_args)
#         sents_with_args.append({"sent": s_with_args, "start_1": arg1_ind, "start_2": arg2_ind, "end_1": arg1_ind_last,
#                                "end_2": arg2_ind_last})
    
#     max_number_of_pairs = 450
#     pairs = list(itertools.product(sents_with_args, repeat=2))
#     print(id, len(pairs))
#     random.shuffle(pairs)
#     for pair in pairs[:max_number_of_pairs]:
#         data.append({"first": pair[0]["sent"], "second": pair[1]["sent"], "query": q, "first_arg1": (pair[0]["start_1"], pair[0]["end_1"]),
#                     "first_arg2": (pair[0]["start_2"], pair[0]["end_2"]), "second_arg1": (pair[1]["start_1"], pair[1]["end_1"]),
#                    "second_arg2": (pair[1]["start_2"], pair[1]["end_2"])})

In [16]:
# len(data)

In [17]:
# random.shuffle(data)
# data[0]

# with open("data.pickle", "wb") as f:
#     pickle.dump(data, f)

In [18]:
with open("data.pickle", "rb") as f:
    data = pickle.load(f)
    
    
with open("data_dev.pickle", "rb") as f:
    data_dev = pickle.load(f)
    
with open("data_train.pickle", "rb") as f:
    data_train = pickle.load(f)

In [19]:
#data = data[:10000]

In [20]:
# import json

# with open("data.txt", "w") as f:
#     for d in data:
#         first, second = d["first"], d["second"]
#         first_arg1 = d["first_arg1"]
#         first_arg2 = d["first_arg2"]
#         second_arg1 = d["second_arg1"]
#         second_arg2 = d["second_arg2"]
        
#         elems = [first, second, first_arg1, first_arg2, second_arg1, second_arg2]
#         keys = ["first", "second", "first_arg1", "first_arg2"]
        
#         f.write(json.dumps(d) + "\n")
#         #f.write(d["first"] + "\t" + d["second"] + "\t" + d["query"] + "\t" + "-".join(d["first_arg1"])+ "\n")

In [21]:
global count
count = 0
global total
total = 0

print(data[1])

{'first': 'The zoonosis caused by <<ARG1:Dirofilaria immitis>> , a parasite of {{ARG2:dogs and other mammals}} , is transmitted by mosquitos and black flies to humans .', 'second': 'N. gonorrhoeae infects columnar cells ; it is therefore a parasite of the cervix , not the vagina .', 'second_with_arguments': 'N. gonorrhoeae infects columnar cells ; <<ARG1:it>> is therefore a parasite of {{ARG2:the cervix}} , not the vagina .', 'query_first': '<E|U+C>arg1:something, $a <E|U+C>arg2:[e]bats $parasite', 'query_second': '<E|U+C>arg1:something $is $a <E|U+C>arg2:[e=ORGANISM|ORGAN|ORGANISM_SUBSTANCE]bats $parasite', 'first_arg1': (4, 5), 'first_arg2': (10, 13), 'second_arg1': (6, 6), 'second_arg2': (12, 13), 'first_arg1_words': ['dogs', 'and', 'other', 'mammals'], 'first_arg2_words': ['dogs', 'and', 'other', 'mammals'], 'second_arg1_words': ['it'], 'second_arg2_words': ['the', 'cervix'], 'query_id': 52}


In [1]:
class BertModel(pl.LightningModule):

    def __init__(self, train_dataset: Dataset, dev_dataset: Dataset, batch_size, device: str, mode: str = "eval"):
        
        super().__init__()
        
        self.device_to_use = device
#         config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased', output_hidden_states=True)
#         self.tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
#         self.model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=config) 
        config = AutoConfig.from_pretrained('finetuned_model/metric_model3', output_hidden_states=True)
        self.tokenizer = AutoTokenizer.from_pretrained('finetuned_model/metric_model3')
        self.model = AutoModel.from_pretrained('finetuned_model/metric_model3', config=config)   
        
        #config = AutoConfig.from_pretrained('Shauli/RE-metric-model-spike', output_hidden_states=True)
        #self.tokenizer = AutoTokenizer.from_pretrained('Shauli/RE-metric-model-spike')
        #self.model = AutoModel.from_pretrained('Shauli/RE-metric-model-spike', config=config)           
        
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.linear_arg1_1 = torch.nn.Linear(768, 64) #torch.load("finetuned_model/metric_model/linear.pt") #torch.nn.Linear(768, 64)
        self.linear_arg1_1.load_state_dict(torch.load("finetuned_model/metric_model3/linear.pt"))
        self.linear_arg2_1 = torch.nn.Linear(768, 64)
        self.linear_arg1_2 = torch.nn.Linear(768, 64)
        self.linear_arg2_2 = torch.nn.Linear(768, 64)
        self.linear_is_same_relation = torch.nn.Linear(768, 1)
        self.linear_is_same_relation.load_state_dict(torch.load("finetuned_model/metric_model3/linear_rel_clf.pt"))
        
        self.bce_loss = torch.nn.BCEWithLogitsLoss()
        
        if mode == "eval":
            
            self.model.eval()
        else:
            self.model.train()
        
        for p in self.model.parameters():
            p.requires_grad = True
        for p in self.model.encoder.layer[-1].parameters():
            p.requires_grad = True
        for p in self.model.encoder.layer[-2].parameters():
            p.requires_grad = True       
        for p in self.model.encoder.layer[-3].parameters():
            p.requires_grad = True    
        for p in self.model.encoder.layer[-4].parameters():
            p.requires_grad = True 
            
        for p in self.model.embeddings.parameters():
            p.requires_grad = True
            
        self.linear_arg1_1.requires_grad = True
        self.linear_arg2_1.requires_grad = True
        self.linear_arg1_2.requires_grad = True
        self.linear_arg2_2.requires_grad = True
        
    
        self.train_gen = torch.utils.data.DataLoader(self.train_dataset, batch_size=batch_size, drop_last=False, shuffle=True,
                                                    num_workers = 4)
        self.dev_gen = torch.utils.data.DataLoader(self.dev_dataset, batch_size=batch_size, drop_last=False, shuffle=False,
                                                  num_workers = 4)
        self.acc = None

        
    def tokenize(self, original_sentence: List[str]) -> Tuple[List[str], Dict[int, int]]:

        """
        Parameters
        ----------
        Returns
        -------
        bert_tokens: The sentence, tokenized by BERT tokenizer.
        orig_to_tok_map: An output dictionary consisting of a mapping (alignment) between indices in the original tokenized sentence, and indices in the sentence tokenized by the BERT tokenizer. See https://github.com/google-research/bert
        """

        bert_tokens = ["[CLS]"]
        orig_to_tok_map = {}
        tok_to_orig_map = {}
        has_subwords = False
        is_subword = []

        for i, w in enumerate(original_sentence):
            tokenized_w = self.tokenizer.tokenize(w)
            has_subwords = len(tokenized_w) > 1
            is_subword.append(has_subwords)
            bert_tokens.extend(tokenized_w)

            orig_to_tok_map[i] = len(bert_tokens) - 1

        tok_to_orig_map = {}

        bert_tokens.append("[SEP]")
        tok_to_orig_map = get_tok_to_orig_map(orig_to_tok_map, len(original_sentence), len(bert_tokens))        
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(bert_tokens)
        tokens_tensor = torch.tensor([indexed_tokens]).to(self.device_to_use)

        return (bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor)
    
    
    def forward(self, x):
        
        outputs = self.model(x)
        states = outputs[0][0] #[seq_len, 768]
        return states
    
    def forward_with_loss_calculation2(self, bert_tokens, x, range_sent1, range_sent2, orig_to_tok_map, l, l_tokens,
                                      metric = "l2", n_max = 5, alpha = 1, mode = "train", normalize=False, nb=0):
        
        range_sent2[0][0] -= l_tokens
        range_sent2[0][1] -= l_tokens
        range_sent2[1][0] -= l_tokens
        range_sent2[1][1] -= l_tokens
        
        idx_arg1_all, idx_arg2_all, all_ngrams = None, None, None
        
        x_1, x_2 = x[0,:l_tokens], x[0,l_tokens:]
        x_1, x_2 = torch.unsqueeze(x_1,0), torch.unsqueeze(x_2,0)
        
        outputs_1, outputs_2 = self.model(x_1), self.model(x_2)
        states_1 = outputs_1[0][0] #[seq_len, 768]
        states_2 = outputs_2[0][0] #[seq_len, 768]
        
        if metric == "cosine" or normalize:
            states_1 = states_1 / (torch.norm(states_1, dim = 1, keepdim = True)+1e-8)
            states_2 = states_2 / (torch.norm(states_2, dim = 1, keepdim = True)+1e-8)
        
        states_1 = self.linear_arg1_1(states_1)
        states_2 = self.linear_arg1_2(states_2)
        states = torch.cat([states_1, states_2], dim = 0)
        
        arg1_sent1, arg2_sent1 = range_sent1
        arg1_sent2, arg2_sent2 = range_sent2
        
        sent1_arg1_vec, sent1_arg2_vec = states_1[arg1_sent1[0]:arg1_sent1[1]].mean(dim=0), states_1[arg2_sent1[0]:arg2_sent1[1]].mean(dim=0)
        sent2_arg1_vec, sent2_arg2_vec = states_2[arg1_sent2[0]:arg1_sent2[1]].mean(dim=0), states_2[arg2_sent2[0]:arg2_sent2[1]].mean(dim=0)        
        
        all_false_ngrams_ranges = get_all_ngrams_spans(len(states_2), [arg1_sent2, arg2_sent2], start_ind = 0,
                                                      n_max = n_max)      
        negatives = [states_2[ngram[0]:ngram[1]].mean(dim=0) for ngram in all_false_ngrams_ranges]
        negatives_arg1 = negatives + [sent2_arg2_vec]
        negatives_arg2 = negatives + [sent2_arg1_vec]
        negatives_arg1 = torch.stack(negatives_arg1).to(self.device_to_use)
        negatives_arg2 = torch.stack(negatives_arg2).to(self.device_to_use)
        

        if mode == "eval":
            all_ngrams = get_all_ngrams_spans(len(states), [], start_ind = l_tokens,
                                                      n_max = n_max)
            ngrams = [states[ngram[0]:ngram[1]].mean(dim=0) for ngram in all_ngrams]
            ngrams = torch.stack(ngrams).to(self.device_to_use)
        
        
        if metric == "l2":
            dists_arg1 = torch.sqrt(((negatives_arg1-sent1_arg1_vec)**2).sum(dim = 1))
            dists_arg2 = torch.sqrt(((negatives_arg2-sent1_arg2_vec)**2).sum(dim = 1))
            dist_arg1_gold = (sent1_arg1_vec - sent2_arg1_vec).norm()
            dist_arg2_gold = (sent1_arg2_vec - sent2_arg2_vec).norm()
            if mode == "eval":
                dist_arg1_all = torch.sqrt(((ngrams-sent1_arg1_vec)**2).sum(dim = 1))
                dist_arg2_all = torch.sqrt(((ngrams-sent1_arg2_vec)**2).sum(dim = 1))
                idx_arg1_all = torch.argsort(dist_arg1_all).detach().cpu().numpy()
                idx_arg2_all = torch.argsort(dist_arg2_all).detach().cpu().numpy()   
                
        elif metric == "cosine":
            dists_arg1 = 1 - negatives_arg1@sent1_arg1_vec.T
            dists_arg2 = 1 - negatives_arg2@sent1_arg2_vec.T
            dist_arg1_gold = 1 - sent1_arg1_vec@sent2_arg1_vec.T
            dist_arg2_gold = 1 - sent1_arg2_vec@sent2_arg2_vec.T
        
        idx_arg1 = torch.argsort(dists_arg1).detach().cpu().numpy()
        idx_arg2 = torch.argsort(dists_arg2).detach().cpu().numpy()
        l = max(int(len(negatives)*0.3),1)
        k = random.choice(range(min(len(negatives), 4))) if np.random.random() < 0.5 else random.choice(range(l))

        dist_arg1_argmax = dists_arg1[idx_arg1[k]]
        dist_arg2_argmax = dists_arg2[idx_arg2[k]]
        
        loss_arg1 = torch.max(torch.zeros(1).to(self.device_to_use), dist_arg1_gold - dist_arg1_argmax + alpha)
        loss_arg2 = torch.max(torch.zeros(1).to(self.device_to_use), dist_arg2_gold - dist_arg2_argmax + alpha)
        
        loss = states[0,0:1]**2 #torch.zeros(1).to(self.device)
        loss2_isnan = np.isnan(loss_arg2.detach().cpu().numpy().item())
        loss1_isnan = np.isnan(loss_arg1.detach().cpu().numpy().item())
        if not loss2_isnan:
            loss += loss_arg2
        if not loss1_isnan:
            loss += loss_arg1
        
        if loss1_isnan or loss2_isnan:
            print("ERROR: nan loss", loss1_isnan, loss2_isnan, nb)
            return
        
        global count
        global total
        total += 1                                                      
        #if loss.detach().cpu().numpy().item() < 1e-5:
        if (dist_arg1_gold - dist_arg1_argmax).detach().cpu().numpy().item() < 0 and (dist_arg2_gold - dist_arg2_argmax).detach().cpu().numpy().item() < 0:
            count += 1
        
        return loss, idx_arg1, idx_arg2, idx_arg1_all, idx_arg2_all, all_false_ngrams_ranges, all_ngrams
        #return loss, np.argsort(dists_arg1+mask_gold_arg1)

    def forward_with_loss_calculation(self, bert_tokens, x, range_sent1, range_sent2, orig_to_tok_map, l, l_tokens,
                                      metric = "l2", n_max = 9, alpha = 0.075, mode = "train", normalize=False, nb=0):
        idx_arg1_all, idx_arg2_all, all_ngrams = None, None, None
        
        outputs = self.model(x)
        states = outputs[0][0] #[seq_len, 768]
        if metric == "cosine" or normalize:
            states = states / (torch.norm(states, dim = 1, keepdim = True)+1e-8)
        
        is_neg_pred = self.linear_is_same_relation(states[0])
        
        states = self.linear_arg1_1(states)
        arg1_sent1, arg2_sent1 = range_sent1
        arg1_sent2, arg2_sent2 = range_sent2
        
        sent1_arg1_vec, sent1_arg2_vec = states[arg1_sent1[0]:arg1_sent1[1]].mean(dim=0), states[arg2_sent1[0]:arg2_sent1[1]].mean(dim=0)
        sent2_arg1_vec, sent2_arg2_vec = states[arg1_sent2[0]:arg1_sent2[1]].mean(dim=0), states[arg2_sent2[0]:arg2_sent2[1]].mean(dim=0)        
        
        all_false_ngrams_ranges = get_all_ngrams_spans(len(states), [arg1_sent1, arg1_sent2, arg2_sent1, arg2_sent2], start_ind = 0,
                                                      n_max = n_max)      
        negatives = [states[ngram[0]:ngram[1]].mean(dim=0) for ngram in all_false_ngrams_ranges]
        negatives_arg1 = negatives + [sent1_arg2_vec, sent2_arg2_vec]
        negatives_arg2 = negatives + [sent1_arg1_vec, sent2_arg1_vec]
        negatives_arg1 = torch.stack(negatives_arg1).to(self.device_to_use)
        negatives_arg2 = torch.stack(negatives_arg2).to(self.device_to_use)
        

        if mode == "eval":
            all_ngrams = get_all_ngrams_spans(len(states), [], start_ind = l_tokens,
                                                      n_max = n_max)
            ngrams = [states[ngram[0]:ngram[1]].mean(dim=0) for ngram in all_ngrams]
            ngrams = torch.stack(ngrams).to(self.device_to_use)
        
        
        if metric == "l2":
            dists_arg1 = torch.sqrt(((negatives_arg1-sent1_arg1_vec)**2).sum(dim = 1))
            dists_arg2 = torch.sqrt(((negatives_arg2-sent1_arg2_vec)**2).sum(dim = 1))
            dist_arg1_gold = (sent1_arg1_vec - sent2_arg1_vec).norm()
            dist_arg2_gold = (sent1_arg2_vec - sent2_arg2_vec).norm()
            if mode == "eval":
                dist_arg1_all = torch.sqrt(((ngrams-sent1_arg1_vec)**2).sum(dim = 1))
                dist_arg2_all = torch.sqrt(((ngrams-sent1_arg2_vec)**2).sum(dim = 1))
                idx_arg1_all = torch.argsort(dist_arg1_all).detach().cpu().numpy()
                idx_arg2_all = torch.argsort(dist_arg2_all).detach().cpu().numpy()   
                
        elif metric == "cosine":
            dists_arg1 = 1 - negatives_arg1@sent1_arg1_vec.T
            dists_arg2 = 1 - negatives_arg2@sent1_arg2_vec.T
            dist_arg1_gold = 1 - sent1_arg1_vec@sent2_arg1_vec.T
            dist_arg2_gold = 1 - sent1_arg2_vec@sent2_arg2_vec.T
        
        idx_arg1 = torch.argsort(dists_arg1).detach().cpu().numpy()
        idx_arg2 = torch.argsort(dists_arg2).detach().cpu().numpy()
        l = max(int(len(negatives)*0.3),1)
        k = random.choice(range(min(len(negatives), 2))) if np.random.random() < 0.5 else random.choice(range(l))

        dist_arg1_argmax = dists_arg1[idx_arg1[k]]
        dist_arg2_argmax = dists_arg2[idx_arg2[k]]
        
        loss_arg1 = torch.max(torch.zeros(1).to(self.device_to_use), dist_arg1_gold - dist_arg1_argmax + alpha)
        loss_arg2 = torch.max(torch.zeros(1).to(self.device_to_use), dist_arg2_gold - dist_arg2_argmax + alpha)
        
        # softmax triplet
        
        z = torch.max(dist_arg1_argmax, dist_arg1_gold)
        temp = 1
        pos_arg1 = torch.exp((dist_arg1_gold - z)/temp)
        neg_arg1 = torch.exp((dist_arg1_argmax - z)/temp)
        loss_arg1 = (pos_arg1 / (pos_arg1 + neg_arg1))#**2

        z = torch.max(dist_arg2_argmax, dist_arg2_gold)
        pos_arg2 = torch.exp((dist_arg2_gold - z)/temp)
        neg_arg2 = torch.exp((dist_arg2_argmax - z)/temp)
        loss_arg2 = (pos_arg2 / (pos_arg2 + neg_arg2))#**2

        
        loss = states[0,0:1]**2 #torch.zeros(1).to(self.device)
        loss2_isnan = np.isnan(loss_arg2.detach().cpu().numpy().item())
        loss1_isnan = np.isnan(loss_arg1.detach().cpu().numpy().item())
        if not loss2_isnan:
            loss += loss_arg2
        if not loss1_isnan:
            loss += loss_arg1
        
        if loss1_isnan or loss2_isnan:
            print("ERROR: nan loss", loss1_isnan, loss2_isnan, nb)
            return
        
        global count
        global total
        total += 1                                                      
        #if loss.detach().cpu().numpy().item() < 1e-5:
        if (dist_arg1_gold - dist_arg1_argmax).detach().cpu().numpy().item() < 0 and (dist_arg2_gold - dist_arg2_argmax).detach().cpu().numpy().item() < 0:
            count += 1
        
        return loss, idx_arg1, idx_arg2, idx_arg1_all, idx_arg2_all, all_false_ngrams_ranges, all_ngrams, is_neg_pred
        #return loss, np.argsort(dists_arg1+mask_gold_arg1)
        
    def training_step(self, batch, batch_nb):
        
        sents_concat, idx, l, sent2_with_args, is_negative = batch
        idx = idx.detach().cpu().numpy()[0]
        bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = self.tokenize(sents_concat[0].split(" "))             

        
        if not is_negative:
            l_tokens = len(bert_tokens[:orig_to_tok_map[l.detach().cpu().numpy().item()-1]]) 
            sent1_range_arg1 = get_entity_range_multiword_expression(idx[0][0], orig_to_tok_map)
            sent1_range_arg2 = get_entity_range_multiword_expression(idx[0][1], orig_to_tok_map)
            sent2_range_arg1 = get_entity_range_multiword_expression(idx[1][0], orig_to_tok_map)
            sent2_range_arg2 = get_entity_range_multiword_expression(idx[1][1], orig_to_tok_map)
            range_sent1 = [sent1_range_arg1, sent1_range_arg2]
            range_sent2 = [sent2_range_arg1, sent2_range_arg2]
            loss, _, _, _, _, _, _, is_neg_pred = self.forward_with_loss_calculation(bert_tokens, tokens_tensor, range_sent1, range_sent2, orig_to_tok_map, l, l_tokens, nb = batch_nb)
        else:
            loss = torch.zeros(1).to(self.device)       
            outputs = self.model(tokens_tensor)
            states = outputs[0][0]
            is_neg_pred = self.linear_is_same_relation(states[0])
        
        y = torch.ones(1).to(self.device) if is_negative else torch.zeros(1).to(self.device)
        loss += self.bce_loss(is_neg_pred, y)
        
#         if np.isnan(loss.detach().cpu().numpy().item()) or loss.detach().cpu().numpy().item() > 1e4:
#             print("ERRROR!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#             print(sents_concat, range_sent1, range_sent2, sent1_idx, sent2_idx)
#             return {"loss": loss*0}

        if total%1000 == 0:
            print("count", count/total)
        return {'loss': loss}
    
    def validation_step(self, batch, batch_nb):

        sents_concat, idx, l, sent2_with_args, is_negative = batch
        if is_negative:
            return {'val_loss': torch.zeros(1).to(self.device)}
        
        idx = idx.detach().cpu().numpy()[0]
        bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = self.tokenize(sents_concat[0].split(" "))
        l_tokens = len(bert_tokens[:orig_to_tok_map[l.detach().cpu().numpy().item()-1]]) 
        sent1_range_arg1 = get_entity_range_multiword_expression(idx[0][0], orig_to_tok_map)
        sent1_range_arg2 = get_entity_range_multiword_expression(idx[0][1], orig_to_tok_map)
        sent2_range_arg1 = get_entity_range_multiword_expression(idx[1][0], orig_to_tok_map)
        sent2_range_arg2 = get_entity_range_multiword_expression(idx[1][1], orig_to_tok_map)
        
        range_sent1 = [sent1_range_arg1,sent1_range_arg2]
        range_sent2 = [sent2_range_arg1,sent2_range_arg2]
        loss, _, _, _, _, _, _, is_neg_pred = self.forward_with_loss_calculation(bert_tokens, tokens_tensor, range_sent1, range_sent2, orig_to_tok_map, l, l_tokens)
        
#         print(sents_concat)
#         print("---------------")
#         print(sent2_with_args)
#         print("------------")
#         print(" ".join(bert_tokens[sent1_range_arg1[0]:sent1_range_arg1[1]]))
#         print(" ".join(bert_tokens[sent1_range_arg2[0]:sent1_range_arg2[1]]))
#         print(" ".join(bert_tokens[sent2_range_arg1[0]:sent2_range_arg1[1]]))
#         print(" ".join(bert_tokens[sent2_range_arg2[0]:sent2_range_arg2[1]]))
#         print("============================================")
        #loss, argsort = self.forward_with_loss_calculation(tokens_tensor, sent1_idx, sent2_idx, orig_to_tok_map, tok_to_orig_map, l, l_tokens)

        return {'val_loss': loss}
    
    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print("Loss is {}".format(avg_loss))
        return {'avg_val_loss': avg_loss}
    
    def configure_optimizers(self):
        #return torch.optim.RMSprop(self.parameters())
        #return torch.optim.ASGD(self.parameters())
        return torch.optim.SGD(self.parameters(), lr=1e-4, momentum=0.5)        
        #return torch.optim.Adam(self.parameters())
    
#     @pl.data_loader
#     def train_dataloader(self):
#         return self.train_gen

#     @pl.data_loader
#     def val_dataloader(self):
#         # OPTIONAL
#         # can also return a list of val dataloaders
#         return self.dev_gen
    

def evaluate_model(dev_dataset, model, max_ngrams = 2, num_examples = 200):
    
    preds = []
    count = 0
    
    for batch in tqdm.tqdm(dev_dataset):
        if count > num_examples: break
        count += 1

        sents_concat, idx, l, sent2_with_args, is_negative = batch
        if is_negative: continue
        idx = idx.detach().cpu().numpy()
        bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = model.tokenize(sents_concat.split(" "))
        l_tokens = len(bert_tokens[:orig_to_tok_map[l-1]]) 
        sent1_range_arg1 = get_entity_range_multiword_expression(idx[0][0], orig_to_tok_map)
        sent1_range_arg2 = get_entity_range_multiword_expression(idx[0][1], orig_to_tok_map)
        sent2_range_arg1 = get_entity_range_multiword_expression(idx[1][0], orig_to_tok_map)
        sent2_range_arg2 = get_entity_range_multiword_expression(idx[1][1], orig_to_tok_map)

        range_sent1 = (sent1_range_arg1,sent1_range_arg2)
        range_sent2 = (sent2_range_arg1,sent2_range_arg2)
        
        loss, idx_arg1, idx_arg2, idx_arg1_all, idx_arg2_all, all_false_ngrams_ranges, all_ngrams, is_neg_pred = model.forward_with_loss_calculation(bert_tokens, tokens_tensor, range_sent1, range_sent2, orig_to_tok_map, l, l_tokens, mode = "eval", n_max=max_ngrams)
        is_neg_pred = torch.sigmoid(is_neg_pred).detach().cpu().numpy().item()
        same_relation_score = 1 - is_neg_pred
        preds.append({"sent": sents_concat, "tokens": bert_tokens, "tok2orig": tok_to_orig_map, "orig2tok": orig_to_tok_map,
                     "preds_arg1_tokens": idx_arg1_all, "preds_arg2_tokens": idx_arg2_all, "false_ngrams": all_false_ngrams_ranges,
                      "all_ngrams": all_ngrams, "gold_arg1_range_tokens": sent2_range_arg1, "gold_arg2_range_tokens": sent2_range_arg2, "same_rel_pred": same_relation_score,
                     "is_negative": is_negative})
        
    return preds


def get_entity_range(index_orig, orig_to_tok_map):
    
    m = min(orig_to_tok_map.keys())
    if orig_to_tok_map[index_orig] == 1: return (1,2)
    if index_orig == 0: return (1, orig_to_tok_map[index_orig] + 1)
    
    before = index_orig - 1
    tok_range = (orig_to_tok_map[before] + 1, orig_to_tok_map[index_orig] + 1)
    return tok_range

def get_entity_range_multiword_expression(start_and_end, orig_to_tok_map):
    
    start, end = start_and_end
    start_range = get_entity_range(start, orig_to_tok_map)
    end_range = get_entity_range(end, orig_to_tok_map)
    return [start_range[0], end_range[1]]

def get_tok_to_orig_map(orig_to_tok_map, num_words, num_tokens):
    
    ranges = [get_entity_range(i, orig_to_tok_map) for i in range(num_words)]
    tok_to_orig_map = {}
    for i in range(num_words):
        min,max = ranges[i]
        for tok in range(min,max):
            tok_to_orig_map[tok] = i
    
    for tok in range(num_tokens):
        if tok not in tok_to_orig_map:
            tok_to_orig_map[tok] = num_words -1
    
    return tok_to_orig_map
        
def get_all_ngrams_spans(seq_len, forbidden_ranges: List[tuple], start_ind = 0, n_max = 15):
    
    def is_intersecting(ngram, forbidden_ranges):
        
        return [(r[1] > ngram[0] >= r[0]) or(r[1] > ngram[1] >= r[0]) for r in forbidden_ranges]
    
    all_ngrams = []
    for n in range(2,n_max+1):
        ngrams = list(get_ngrams(range(start_ind, seq_len), n))
        all_ngrams.extend(ngrams)
    
    all_ngrams = [(ngram[0], ngram[-1]) for ngram in all_ngrams]
    all_ngrams = [ngram for ngram in all_ngrams if not any(is_intersecting(ngram, forbidden_ranges))]
    return all_ngrams
    
    
def get_prediction(sent1, sent2, model):
    
    l = len(sent1.split(" ")) + 1 
    sent2 = sent2.replace("ARG1:", "").replace("ARG2:", "")
    
    sents_concat = sent1 + " ***** " + sent2
    sent1_arg1 = [i for i,w in enumerate(sent1.split(" ")) if "ARG1:" in w][0]
    sent1_arg2 = [i for i,w in enumerate(sent1.split(" ")) if "ARG2:" in w][0]
    words = sents_concat.split(" ")
    
    bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = model.tokenize(sents_concat.split(" "))
    range_arg1 = get_entity_range(sent1_arg1, orig_to_tok_map)
    range_arg2 = get_entity_range(sent1_arg2, orig_to_tok_map)
    
    l_tokens = len(bert_tokens[:orig_to_tok_map[l-1]])    
    outputs = model.model(tokens_tensor)
    states = outputs[0][0] #[seq_len, 768]

    states = states#/torch.norm(states, dim = 1, keepdim = True)
    
    sent1_arg1_vec, sent1_arg2_vec = states[range_arg1[0]:range_arg1[1]].mean(dim=0), states[range_arg2[0]:range_arg2[1]].mean(dim=0)
    
    print("GOLD ARG1, ARG2:", bert_tokens[orig_to_tok_map[sent1_arg1]], bert_tokens[orig_to_tok_map[sent1_arg2]])
    sims_arg1 = (model.linear_arg1_1(states)-model.linear_arg1_2(sent1_arg1_vec)).norm(dim=1)
    sims_arg2 = (model.linear_arg2_1(states)-model.linear_arg2_2(sent1_arg2_vec)).norm(dim=1)

    mask_gold_arg1 = torch.zeros_like(sims_arg1).to(model.device)
    mask_gold_arg2 = torch.zeros_like(sims_arg2).to(model.device)
    #mask_gold_arg1[range_arg1[0]:range_arg1[1]] = 1e6
    #mask_gold_arg2[range_arg2[0]:range_arg2[1]] = 1e6
    mask_gold_arg2[:l_tokens] = 1e6
    mask_gold_arg1[:l_tokens] = 1e6
        
    idx_arg1 = torch.argsort(sims_arg1+mask_gold_arg1).detach().cpu().numpy()
    idx_arg2 = torch.argsort(sims_arg2+mask_gold_arg2).detach().cpu().numpy()
    
    print("all arg1 preds", [words[tok_to_orig_map[idx_arg1[i]]] for i in range(len(idx_arg1))][:7])
    print("---------------------------------------------------")
    print("all arg2 preds", [words[tok_to_orig_map[idx_arg2[i]]] for i in range(len(idx_arg2))][:7])
    print("---------------------------------------------------")
    
    #print(bert_tokens[idx_arg1[-4]], bert_tokens[idx_arg2[-4]])
    
    return
    
    most_sim_arg1, most_sim_arg2 = sims_arg1[idx_arg1[-2]], sims_arg2[idx_arg2[-2]]
    ind_arg1, ind_arg2 = idx_arg1[-2].detach().cpu().numpy().item(), idx_arg2[-2].detach().cpu().numpy().item()
    ind_arg1, ind_arg2 = ind_arg1 + l_tokens, ind_arg2 + l_tokens
    
    return sents_concat, tok_to_orig_map[ind_arg1], tok_to_orig_map[ind_arg2], bert_tokens[ind_arg1], bert_tokens[ind_arg2]

NameError: name 'pl' is not defined

In [23]:
class Dataset(torch.utils.data.Dataset):
    """Simple torch dataset class"""

    def __init__(self, data: List[Dict], device = "cpu"):

        self.data = data
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        with torch.no_grad():
            
            d = self.data[index]
            other = random.choice(range(len(self)))

            if random.random() < 1 or (self.data[other]["query_id"] == d["query_id"]):
                sent1, sent2 = d["first"], d["second"]
                is_negative = False
            else:
                sent1 = d["first"]
                sent2 = self.data[other]["second"] 
                is_negative = True
                
            sent1_arg1, sent1_arg2 = list(d["first_arg1"]), list(d["first_arg2"])
            sent2_arg1, sent2_arg2 = list(d["second_arg1"]), list(d["second_arg2"])            
            l = len(sent1.split(" ")) + 1 
            #sent2_arg1, sent2_arg2 = sent2_arg1 + l, sent2_arg2 + l
            sent2_arg1[0] += l
            sent2_arg1[1] += l
            sent2_arg2[0] += l
            sent2_arg2[1] += l

            sent2 = sent2.replace("ARG1:", "").replace("ARG2:", "")
            sents_concat = sent1 + " ***** " + sent2 #sents_concat.split(" ")[l] is the first token in the 2nd sent
            #create idx tensor. # 1stdim: sents, 2st dim: arg, 3st dim: start and end
            idx = [[[sent1_arg1[0], sent1_arg1[1]], [sent1_arg2[0], sent1_arg2[1]]], [[sent2_arg1[0], sent2_arg1[1]], [sent2_arg2[0], sent2_arg2[1]]] ] 
            sent2_with_args = d["second_with_arguments"]
            return sents_concat, torch.tensor(idx).int(), l, sent2_with_args, is_negative

In [24]:
l = int(0.8 * len(data))
#train_dataset, dev_dataset = Dataset(data[:l], "cpu"), Dataset(data[l:], "cpu")
train_dataset, dev_dataset = Dataset(data_train, "cpu"), Dataset(data_dev, "cpu")

sents_concat, idx, l, sent2_with_args, is_negative = train_dataset[1]
print(sents_concat)
print(idx, idx.shape)
print(is_negative)

The zoonosis caused by <<ARG1:Dirofilaria immitis>> , a parasite of {{ARG2:dogs and other mammals}} , is transmitted by mosquitos and black flies to humans . ***** N. gonorrhoeae infects columnar cells ; it is therefore a parasite of the cervix , not the vagina .
tensor([[[ 4,  5],
         [10, 13]],

        [[32, 32],
         [38, 39]]], dtype=torch.int32) torch.Size([2, 2, 2])
False


In [25]:
try:
    del model
except:
    pass
model = BertModel(train_dataset, dev_dataset, 1, "cuda", "train")

In [ ]:
trainer = Trainer(max_epochs=50,min_epochs=1, gpus = 1)
trainer.fit(model, model.train_gen, model.dev_gen)

GPU available: True, used: True
2020-11-23 09:16:38.884 INFO    lightning: GPU available: True, used: True
TPU available: False, using: 0 TPU cores
2020-11-23 09:16:38.887 INFO    lightning: TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2020-11-23 09:16:38.890 INFO    lightning: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                    | Type              | Params
--------------------------------------------------------------
0 | model                   | BertModel         | 109 M 
1 | linear_arg1_1           | Linear            | 49 K  
2 | linear_arg2_1           | Linear            | 49 K  
3 | linear_arg1_2           | Linear            | 49 K  
4 | linear_arg2_2           | Linear            | 49 K  
5 | linear_is_same_relation | Linear            | 769   
6 | bce_loss                | BCEWithLogitsLoss | 0     
2020-11-23 09:16:40.098 INFO    lightning: 
  | Name                    | Type              | Params
--------------------

count 0.699
count 0.7
count 0.7043333333333334
count 0.7115
count 0.713
count 0.716
count 0.7227142857142858
count 0.72475
count 0.7264444444444444


In [ ]:
# #model.to("cpu")
# #model.device = "cpu"
# while True:
#     try:
#         trainer = Trainer(max_nb_epochs=50,min_nb_epochs=1, gpus = 1)
#         trainer.fit(model)
#     except Exception as e:
#         continue

In [ ]:
model.model.save_pretrained("finetuned_model/metric_model3")
model.tokenizer.save_pretrained("finetuned_model/metric_model3")
torch.save(model.linear_arg1_1.state_dict(), 'finetuned_model/metric_model3/linear.pt', _use_new_zipfile_serialization = False)
torch.save(model.linear_is_same_relation.state_dict(), 'finetuned_model/metric_model3/linear_rel_clf.pt', _use_new_zipfile_serialization = False)

## model.model.layers

In [ ]:
preds = evaluate_model(dev_dataset, model.cuda(), max_ngrams = 8, num_examples = 1000)

In [ ]:
preds[0].keys()

In [ ]:
def pretty_print(sent, idx_arg1, idx_arg2):
    
    sent_lst = sent.split(" ")
    sent = " ".join(sent_lst[:idx_arg1[0]]) + " " + colored(" ".join(sent_lst[idx_arg1[0]:idx_arg1[1]]), "red") + " " + " ".join(sent_lst[idx_arg1[1]:])
    sent_lst = sent.split(" ")
    sent = " ".join(sent_lst[:idx_arg2[0]]) + " " + colored(" ".join(sent_lst[idx_arg2[0]:idx_arg2[1]]), "blue") + " " + " ".join(sent_lst[idx_arg2[1]:])
    return sent

good, bad = 0., 0.

def is_contained(pred, gold):
    return pred[0] >= gold[0] and pred[1] <= gold[1]

for p in preds:
    sent = p["sent"]
    tokens = p["tokens"]
    range_arg1, range_arg2 = p["gold_arg1_range_tokens"], p["gold_arg2_range_tokens"]
    
    print(colored("Is negative: {}".format(p["is_negative"], "black")))
    print("Same relation pred: {}".format(p["same_rel_pred"]))
    print("Gold arg1: {}; Gold arg2: {}".format(" ".join(tokens[range_arg1[0]:range_arg1[1]]), " ".join(tokens[range_arg2[0]:range_arg2[1]])))
    pred_arg1, pred_arg2 = p["preds_arg1_tokens"], p["preds_arg2_tokens"]
    ngram_pred_arg1_idx, ngram_pred_arg2_idx = p["all_ngrams"][pred_arg1[0]], p["all_ngrams"][pred_arg2[0]]
    print("Pred arg1: {}; Pred arg2: {}".format(" ".join(tokens[ngram_pred_arg1_idx[0]:ngram_pred_arg1_idx[1]]),
         " ".join(tokens[ngram_pred_arg2_idx[0]:ngram_pred_arg2_idx[1]])))
    
    arg1_start = p["tok2orig"][ngram_pred_arg1_idx[0]]
    arg1_end = p["tok2orig"][ngram_pred_arg1_idx[1]]
    arg2_start = p["tok2orig"][ngram_pred_arg2_idx[0]]
    arg2_end = p["tok2orig"][ngram_pred_arg2_idx[1]]
    #print(p["preds_arg1_tokens"][:] == p["preds_arg2_tokens"][:])
    print(pretty_print(sent, [arg1_start, arg1_end], [arg2_start, arg2_end]))
    
    
    #if range_arg1[0] == ngram_pred_arg1_idx[0] and range_arg1[1] == ngram_pred_arg1_idx[1] and range_arg2[0] == ngram_pred_arg2_idx[0] and range_arg2[1] == ngram_pred_arg2_idx[1]:
    correct = is_contained(ngram_pred_arg1_idx, range_arg1) and is_contained(ngram_pred_arg2_idx, range_arg2)
    if correct:    
        good += 1
    else:
        bad += 1
        
    print(colored("Correct: {}".format(correct), "green" if correct else "red"))
    print("=================================================================")
print(good / (good + bad))

In [ ]:
ngram_pred_arg1_idx

In [ ]:
k = random.choice(range(100))

sent1 = "The use of 5 % <<ARG1:dextrose>> and sodium bicarbonate is an effective initial treatment for <<ARG2:hyperkalemia>> ."
sent2 = "Cyclosporine is an immunosuppressant and is used to avoid organ transplant rejection ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())

print("===================================================")
    
    
sent1 = "Suppose that a fraction <<ARG1:Q>> of all infected individuals is completely isolated and does not transmit the disease to <<ARG2:anyone>> ."
sent2 = "Although the J class is isolated to a negative-pressure room , a few of its members could transmit the virus , by accident , to other people such as healthcare workers ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    
    
sent1 = "Moreover , the <<ARG1:pathogen>> also spreads to humans through direct <<ARG2:contact with infected poultry>> and contaminated surfaces [ 5 ] ."
sent2 = "The COVID-19 disease can spread in a population through infected symptomatic/asymptomatic individuals who come in contact directly/indirectly [ 4 ] ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    
    
sent1 = "The low second trimester plasma MBL <<ARG1:level>> is not a risk factor for the development of <<ARG2:preterm birth>> ."
sent2 = "Nevertheless , even 5 - 10 years intervals from the previous gestation do not increase the risk for PE ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    
    
sent1 = "However , <<ARG1:glucose blood levels>> were reduced and insulin sensitivity was increased in <<ARG2:MTX2>> and MTX4 ."
sent2 = "Both blood glucose and lipid profiles were reduced in the allogenic group [ 9 ] ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    
    
sent1 = "It seems likely that the prevention and treatment of <<ARG1:AEx of IPF>> must focus on both <<ARG2:disease-specific>> ( e.g. , anti-fi brotic therapies ) and non-disease-specific ( e.g. , vaccination , prevention of stress ) areas ."
sent2 = "Thus , the clinical management of SARS should consider not only antiviral but anti-inflammatory strategies as well ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
    
print("===================================================")
    
    
sent1 = "The results indicate that supplementation of fish with <<ARG1:rosemary>> could improve the haematological and immunological properties and increase the survival rate after challenge with <<ARG2:S. iniae>> ."
sent2 = "Recent studies have demonstrated that prolonged treatment with EP can ameliorate experimental ulcerative colitis and slow multiple tumor growth ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    
    
sent1 = "In 2013 , an outbreak of <<ARG1:MERS-CoV>> occurred in the <<ARG2:Middle East>> , including Jordan ( 10 , 11 ) ."
sent2 = "An outbreak of severe acute respiratory syndrome ( SARS ) was detected in Singapore at the beginning of March 2003 ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    

for i in range(25):

    sent1 = data[i]["first"]
    sent2 = data[i]["second"].replace("ARG1:", "").replace("ARG2:","")
    with torch.no_grad():
        print(sent1)
        print("-------------------------------")
        print(data[i]["second_with_arguments"])
        print("-------------------------------")
        get_prediction(sent1, sent2, model.eval())

In [ ]:
#del model

In [ ]:
torch.randn(1)*0

In [ ]:
data[0]

In [ ]:

sent = train_dataset[0][0]
bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = model.tokenize(sent.split(" "))
indexed_tokens = model.tokenizer.convert_tokens_to_ids(bert_tokens)
tokens_tensor = torch.tensor([indexed_tokens])

for d in train_dataset:
    sent = d[0]
    bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = model.tokenize(sent.split(" "))
    print(len(bert_tokens))

In [ ]:
train_dataset[0][0]

In [ ]:
print(sent)
print(bert_tokens)
print(get_entity_range((1,3), orig_to_tok_map))

In [ ]:
from nltk import ngrams
list(ngrams(range(10), 3))

In [ ]:
q = get_all_ngrams_spans(10, [(3,5)], start_ind = 0, n_max=2)
q

In [ ]:
q[0]

In [ ]:
states = torch.randn(500,300)
ngrams = get_all_ngrams_spans(len(states), [[0,4], [13, 16]], start_ind = 50)
negatives = [states[ngram[0]:ngram[1]].mean(dim=0) for ngram in ngrams]
negatives = torch.stack(negatives)

In [ ]:
negatives.shape, len(ngrams)

In [ ]:
list(range(2,2))

In [ ]:
model.training_step([train_dataset[0]],0)

In [ ]:
train_dataset[0]

In [ ]:
type(torch.zeros(1).item())

In [ ]:
model.linear_adrg1_1.bias[0:1]

In [ ]:
s,idx,l,_ = train_dataset[1]
idx = idx.detach().cpu().numpy()
s.split(" ")[idx[1][0][0]:idx[1][0][1]+1]
print(s.split(" ")[26:])

In [ ]:
bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = model.tokenize(s.split(" "))
sent1_range_arg1 = get_entity_range_multiword_expression(idx[0][0], orig_to_tok_map)
sent1_range_arg2 = get_entity_range_multiword_expression(idx[0][1], orig_to_tok_map)
sent2_range_arg1 = get_entity_range_multiword_expression(idx[1][0], orig_to_tok_map)
sent2_range_arg2 = get_entity_range_multiword_expression(idx[1][1], orig_to_tok_map)
l_tokens = orig_to_tok_map[l]

In [ ]:
sent2_range_arg1
bert_tokens_2 = bert_tokens[l_tokens:]
bert_tokens_2[sent2_range_arg1[0]-l_tokens:sent2_range_arg1[1]-l_tokens]

In [ ]:
l_tokens

In [ ]:
AutoModel.from_pretrained('Shauli/RE-metric-model-spike')

In [ ]:
config = AutoConfig.from_pretrained('Shauli/IE-metric-model-spike', output_hidden_states=True)
model = AutoModel.from_pretrained('Shauli/RE-metric-model-spike', config=config)  

In [ ]:
config = AutoConfig.from_pretrained('Shauli/IE-metric-model-spike', output_hidden_states=True)

In [ ]:
train_dataset[0]

In [8]:
import torch
torch.cat([torch.zeros(2,3), torch.zeros(4,3)], dim = 0)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])